In [77]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter  
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
#import weaviate # type: ignore
from weaviate.embedded import EmbeddedOptions # type: ignore
from dotenv import load_dotenv,find_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate # type: ignore
#from weaviate.embedded import EmbeddedOptions # type: ignore
from dotenv import load_dotenv,find_dotenv
# 
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


In [78]:
# Data loader
def data_loader(file_path= '../prompts/context.txt'):
    loader = TextLoader(file_path)
    documents = loader.load()

    # Chunk the data
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    return chunks



In [79]:
from langchain_community.vectorstores.chroma import Chroma

def create_retriever(chunks):
    # Load OpenAI API key from .env file
    load_dotenv(find_dotenv())

    # Setup Weaviate client
    weaviate_client = weaviate.Client(embedded_options=EmbeddedOptions())

    # Populate vector database with Weaviate
    vectorstore = Weaviate.from_documents(
        client=weaviate_client,
        documents=chunks,
        embedding=OpenAIEmbeddings(),
        by_text=False
    )

    # Define vectorstore as retriever to enable semantic search
    retriever = vectorstore.as_retriever()
    return retriever


In [80]:
chunks =  data_loader()
chunks

[Document(page_content='In recent years, natural language processing (NLP) has seen significant advancements, \ndriven by the development of large-scale language models such as GPT (Generative Pre-trained \nTransformer) and BERT (Bidirectional Encoder Representations from Transformers). \nThese models, trained on vast amounts of text data, have demonstrated remarkable capabilities \nin tasks such as language generation, text summarization, sentiment analysis, \nand question answering.\nOne area where these models have shown promise is in facilitating human-computer \ninteraction through conversational interfaces. Chatbots, virtual assistants, \nand automated customer service systems leverage NLP models to understand user queries, \nprovide relevant information, and engage in natural language conversations.\nHowever, generating prompts that effectively guide the behavior of these models remains \na challenge. Prompts play a crucial role in shaping the responses generated by language mod

In [81]:
base_retriever = create_retriever(chunks)

/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


embedded weaviate is already listening on port 8079


{"level":"info","msg":"Created shard langchain_634db4ce942c454ca1959fafd9440577_jcqEpAIJPHLS in 1.174115ms","time":"2024-06-06T16:38:40+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-06-06T16:38:40+03:00","took":173526}
/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [82]:
template = """You are an AI-powered natural language processing expert in information retrieval and ranking. Your role is to provide advanced techniques and algorithms for generating superior prompts that optimize user queries and ensure the best performance of automatic prompt generation. Your expertise lies in understanding user intent, analyzing query patterns, and generating contextually relevant prompts that enable efficient and accurate retrieval of information. With your skills and abilities, you are capable of fine-tuning models to enhance prompt generation, leveraging semantic understanding and query understanding to deliver optimal results. By utilizing cutting-edge techniques in the field, you can generate automatic prompts that empower users to obtain the most relevant and comprehensive information for their queries.

Your task is to formulate exactly {num_of_prompts_to_generate} prompts from the provided original prompt that are better and using the given context.

Use the below format to output the prompts.

example:
["prompt1", "prompt2", "prompt3", "prompt4", "prompt5"]


The generated prompt must satisfy the rules given below:
0. The generated prompted should only contain the prompt and no numbering
1.The prompt should make sense to humans even when read without the given context.
2.The prompt should be fully created from the given context.
3.The prompt should be framed from a part of context that contains important information. It can also be from tables,code,etc.
4.The prompt must be reasonable and must be understood and responded by humans.
5.Do no use phrases like 'provided context',etc in the prompt
6.The prompt should not contain more than 10 words, make of use of abbreviation wherever possible.
    
### CONTEXT
{context}

### User Prompt
User Prompt: {user_prompt}
"""


In [83]:
prompt = ChatPromptTemplate.from_template(template)

In [84]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough, RunnableParallel

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retriever =  RunnableParallel({"context": itemgetter("user_prompt") | base_retriever, "user_prompt":itemgetter('user_prompt'), "num_of_prompts_to_generate":itemgetter("num_of_prompts_to_generate"),})

retrieval_augmented_qa_chain = retriever | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}


In [85]:
import json
user_prompt = "What is model?"
num_of_prompts_to_generate =5
result = retrieval_augmented_qa_chain.invoke({"user_prompt":user_prompt, "num_of_prompts_to_generate":num_of_prompts_to_generate})
print(result)
prompts_generated = json.loads(result["response"].content)
prompts_generated

/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


{'response': AIMessage(content='["What are GPT and BERT?", "How do language models work?", "Why are prompts important for language models?", "How do NLP systems interact with users?", "What advancements have been made in NLP?"]', response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 657, 'total_tokens': 703}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2c319cfd-7207-4999-a521-265f58ce9cdc-0', usage_metadata={'input_tokens': 657, 'output_tokens': 46, 'total_tokens': 703}), 'context': [Document(page_content='In recent years, natural language processing (NLP) has seen significant advancements, \ndriven by the development of large-scale language models such as GPT (Generative Pre-trained \nTransformer) and BERT (Bidirectional Encoder Representations from Transformers). \nThese models, trained on vast amounts of text data, have demonstrated remarkable capabilities \nin tasks such as language generation

['What are GPT and BERT?',
 'How do language models work?',
 'Why are prompts important for language models?',
 'How do NLP systems interact with users?',
 'What advancements have been made in NLP?']

Ground Truth Creation

In [86]:
def get_context_for_user_objective(user_objective):
    return base_retriever.get_relevant_documents(user_objective)

In [87]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="questions",
    description="list of questions about the context with the example: ['What is model'].",
    type="array(str)"
)

question_response_schemas = [
    question_schema,
]

question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [89]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a University Professor creating a test for advanced students. For each context, create 5 question that is specific to the context. Avoid creating generic or general questions.

Format the output as JSON with the following keys:
questions:

Format the output as the following:
questions: [
    "Question 1",
    "Question 2"
]

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=get_context_for_user_objective(user_prompt),
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
questions_dict = question_output_parser.parse(response.content)
print(questions_dict)

/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


{'questions': ['What are some advancements in natural language processing?', 'Which language models have contributed to the advancements in NLP?', 'What are some tasks that language models like GPT and BERT can perform?', 'How do NLP models facilitate human-computer interaction?', 'Why are prompts important in shaping the responses of language models?'], 'context': [{'page_content': 'In recent years, natural language processing (NLP) has seen significant advancements, driven by the development of large-scale language models such as GPT (Generative Pre-trained Transformer) and BERT (Bidirectional Encoder Representations from Transformers). These models, trained on vast amounts of text data, have demonstrated remarkable capabilities in tasks such as language generation, text summarization, sentiment analysis, and question answering. One area where these models have shown promise is in facilitating human-computer interaction through conversational interfaces. Chatbots, virtual assistants,


Answer each question generated using GPT-4 that will act as the ground truth


In [92]:

from tqdm import tqdm

answer_generation_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)
answer_generation_chain = bare_template | answer_generation_llm

question_answer_dict_list  = []
for question in tqdm(questions_dict['questions']):
    print(question)
    messages = prompt_template.format_messages(
        context=get_context_for_user_objective(user_prompt),
        question=question,
        format_instructions=format_instructions
    )

    response = answer_generation_chain.invoke({"content" : messages})
    try:
        output_dict = answer_output_parser.parse(response.content)
    except Exception as e:
        continue
    question_answer_dict_list.append({'question': output_dict["question"],'answer':output_dict["answer"]})
are_template |
question_answer_dict_list
for question in tqdm(questions_dict['questions']):
    print(question)
    messages = prompt_template.format_messages(
        context=get_context_for_user_objective(user_prompt),
        question=question,
        format_instructions=format_instructions
    )
'''
    response = answer_generation_chain.invoke({"content" : messages})
    try:
        output_dict = answer_output_parser.parse(response.content)
    except Exception as e:
        continue
    question_answer_dict_list.append({'question': output_dict["question"],'answer':output_dict["answer"]})

question_answer_dict_list
'''

  0%|          | 0/5 [00:00<?, ?it/s]

What are some advancements in natural language processing?


/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
 20%|██        | 1/5 [00:07<00:31,  7.94s/it]

Which language models have contributed to the advancements in NLP?


/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
 40%|████      | 2/5 [00:13<00:19,  6.64s/it]

What are some tasks that language models like GPT and BERT can perform?


/home/mahbubah/Desktop/week-7/Prompt-Tuning-Precision-RAG/venv/lib/python3.10/site-packages/pydantic/main.py:1070: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
 40%|████      | 2/5 [00:16<00:24,  8.29s/it]


KeyError: 'question'

In [ ]:
from tqdm import tqdm

# Block removed from here

question_answer_dict_list = []
for question in tqdm(questions_dict['questions']):
    print(question)
    messages = prompt_template.format_messages(
        context=get_context_for_user_objective(user_prompt),  # Make sure user_prompt is defined
        question=question,
        format_instructions=format_instructions
    )

    response = answer_generation_chain.invoke({"content": messages})
    try:
        output_dict = answer_output_parser.parse(response.content)
        question_answer_dict_list.append({'question': output_dict["question"], 'answer': output_dict["answer"]})
    except Exception as e:
        # Handle specific exceptions if necessary
        continue

question_answer_dict_list
